In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load the data
file_path = r"C:\Users\jaspe\OneDrive - TU Eindhoven\TUe\Master\Year 1\Q4\Advanced Programming\Assignment 4 group\sorted_tested_molecules.csv"
df = pd.read_csv(file_path)

# Select the first 10 columns for features
X = df.iloc[:, :40]
y = df[['PKM2_inhibition', 'ERK2_inhibition']]

# Convert data to float32
X = X.astype(np.float32)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = y.astype(np.float32)

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_clf = grid_search.best_estimator_

# Train the model using the training sets
best_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Adding the actual and predicted values to a new dataframe with molecule names
results_df = pd.DataFrame({
    'PKM2_actual': y_test['PKM2_inhibition'].values,
    'ERK2_actual': y_test['ERK2_inhibition'].values,
    'PKM2_predicted': y_pred[:, 0],
    'ERK2_predicted': y_pred[:, 1]
})

# Save to new CSV
output_file_path = "predicted_molecules.csv"
results_df.to_csv(output_file_path, index=False)

# Feature importances
feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)

# Plotting feature importances
plt.figure(figsize=(10, 6))
feature_imp.head(10).plot(kind='bar')
plt.title('Top 10 Feature Importances')
plt.ylabel('Importance Score')
plt.show()


Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\jaspe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
171 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jaspe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jaspe\anaconda3\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\jaspe\anaconda3\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\jaspe\anaconda3\lib\site-packag

Accuracy: 0.921146953405018


AttributeError: 'numpy.ndarray' object has no attribute 'columns'